In [1]:
import pandas as pd
from scripts.data_genertion.consts import *

Added a column of the true length of the sequence according to the string length itself

Removed any rows with NaN inhibition cells

In [8]:
df = pd.read_csv('data_updated_18.5.csv')
df['true_length_of_seq'] = df[SEQUENCE].apply(len)
df = df.dropna(subset=[INHIBITION])

/tmp/ipykernel_6710/2826274691.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_updated_18.5.csv')


Filling Nan values

In [9]:
df_filled = df.copy()
for col in df.columns:
    if df[col].dtype == 'object':
        df_filled[col] = df[col].fillna("")
    else:
        df_filled[col] = df[col].fillna(0)

This part identify rows that may be the same experiment but with different modifications - in the type, location or pattern. This checks where there are groups of the same 'experiment_keys' that indicate the experiment, and search for more than one unique value at the 'columns_allowed_to_differ' where the modifications or linkage may be different to enable a comparison for only sequences that we can differentiate without considering the modifications. The added column is 'mod_scan', where 1 is a row that has at least one more row with the same experiment values and different modifications or locations of those.

In [11]:
experiment_keys = [SEQUENCE, 'Density(cells/well)', 'Transfection', 'Treatment_Period(hours)', 'ASO_volume(nM)']

columns_allowed_to_differ = [MODIFICATION, 'Location', 'Chemical_Pattern', 'Linkage', 'Linkage_Location', 'Smiles',
                             INHIBITION]

columns_to_compare = [col for col in df_filled.columns if col not in experiment_keys + columns_allowed_to_differ + ['ISIS', 'index']]

result_temp = []
for keys, group in df_filled.groupby(experiment_keys):
    # Check if any of the modification-related columns differ
    if group[[MODIFICATION, 'Location', 'Chemical_Pattern', 'Linkage', 'Linkage_Location', 'Smiles']].nunique().gt(1).any():
        # Check that all other fields (except inhibition + modifications) are the same
        compare = group[columns_to_compare].reset_index(drop=True)
        if compare.nunique().eq(1).all():
            result_temp.append(group)

if result_temp:
    final_df = pd.concat(result_temp)

    # Get the row indices of matched rows
    mod_scan_indices = final_df.index

    # Add new column to original dataframe
    df_filled['mod_scan'] = 0
    df_filled.loc[mod_scan_indices, 'mod_scan'] = 1

    print(df_filled[df_filled['mod_scan'] == 1])  # Optional: preview marked rows
else:
    df_filled['mod_scan'] = 0
    print("No matching ASO experiment groups with only modification-related differences.")


       index    ISIS Target_gene Cell_line  Density(cells/well) Transfection  \
2          2  540806       K-RAS      A431               5000.0  free uptake   
13        13  651966       K-RAS      A431               5000.0  free uptake   
15        15  651987       K-RAS      A431               5000.0  free uptake   
30        30  695867       K-RAS      A431               5000.0  free uptake   
31        31  695912       K-RAS      A431               5000.0  free uptake   
...      ...     ...         ...       ...                  ...          ...   
36867  36867  972044       APOL1      A431              11000.0  free uptake   
36869  36869  972096       APOL1      A431              11000.0  free uptake   
36872  36872  969448       APOL1      A431              11000.0  free uptake   
36873  36873  971969       APOL1      A431              11000.0  free uptake   
36875  36875  972189       APOL1      A431              11000.0  free uptake   

       ASO_volume(nM)  Treatment_Period

In [13]:
df_filled.to_csv('data_asoptimizer_updated.csv', index=False)

Adding a column of correct cell line to fix A431/A-431

In [12]:
df_org = pd.read_csv('data_from_article_fixed.csv')
df_ver2 = pd.read_csv('data_updated_18.5.csv')
df_ver3 = pd.read_csv('data_asoptimizer_updated.csv')

/tmp/ipykernel_1753/2373224261.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ver2 = pd.read_csv('data_updated_18.5.csv')
/tmp/ipykernel_1753/2373224261.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ver3 = pd.read_csv('data_asoptimizer_updated.csv')


In [13]:
df_org['cell_line_uniform'] = df_org['Cell_line']
df_org.loc[(df_org['Cell_line'] == 'A431') | (df_org['Cell_line'] == 'A-431'), 'cell_line_uniform'] = 'A431'

In [15]:
df_ver2['cell_line_uniform'] = df_ver2['Cell_line']
df_ver2.loc[(df_ver2['Cell_line'] == 'A431') | (df_ver2['Cell_line'] == 'A-431'), 'cell_line_uniform'] = 'A431'

In [16]:
df_ver3['cell_line_uniform'] = df_ver3['Cell_line']
df_ver3.loc[(df_ver3['Cell_line'] == 'A431') | (df_ver3['Cell_line'] == 'A-431'), 'cell_line_uniform'] = 'A431'

In [22]:
df_org.to_csv('data_from_article_fixed.csv', index=False)
df_ver2.to_csv('data_updated_18.5.csv', index=False)
df_ver3.to_csv('data_asoptimizer_updated.csv', index=False)

Adding correcting inhibition column and mutual index column for identical rows

In [26]:
df_org = pd.read_csv('data_asoptimizer_updated.csv')

/tmp/ipykernel_11479/2090144437.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_org = pd.read_csv('data_asoptimizer_updated.csv')


In [72]:
df_org['Transcript'] = df_org['Transcript'].fillna(0)
df_org['Transcript'] = df_org['Transcript'].astype(str)
df_org['Transfection'] = df_org['Transfection'].fillna(0)
df_org['Transfection'] = df_org['Transfection'].astype(str)
df_org['Primer_probe_set'] = df_org['Primer_probe_set'].fillna(0)
df_org['Primer_probe_set'] = df_org['Primer_probe_set'].astype(str)

In [73]:
def process_duplicate_inhibition(df):
    """
    Process dataframe to handle duplicate rows (identical except for inhibition and index).

    Parameters:
    df (pd.DataFrame): Input dataframe with columns including 'inhibition' and 'index'

    Returns:
    pd.DataFrame: Processed dataframe with 'corrected_inhibition' and 'mutual_index' columns
    """

    # Create a copy to avoid modifying the original dataframe
    result_df = df.copy()

    # Get all columns except 'inhibition' and 'index' for grouping
    grouping_cols = [col for col in df.columns if col not in [INHIBITION, 'index']]

    # Group by all columns except inhibition and index
    grouped = df.groupby(grouping_cols)

    # Initialize the new columns
    result_df['corrected_inhibition'] = df[INHIBITION]
    result_df['mutual_index'] = df['index']

    # Process each group
    for name, group in grouped:
        if len(group) > 1:  # This group has duplicates
            # Calculate average inhibition for duplicates
            avg_inhibition = group[INHIBITION].mean()

            # Get the first index as the mutual index
            mutual_idx = group['index'].iloc[0]

            # Update all rows in this group
            mask = df.index.isin(group.index)
            result_df.loc[mask, 'corrected_inhibition'] = avg_inhibition
            result_df.loc[mask, 'mutual_index'] = mutual_idx

    return result_df

In [74]:
df_copy = process_duplicate_inhibition(df_org)


In [75]:
df_copy

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,corrected_inhibition,mutual_index
0,0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,...,KRAS,human,0,41212.0,0.901972,16,0,A431,33.0,0
1,1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,...,KRAS,human,0,23686.0,0.518395,16,0,A431,7.0,1
2,2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,...,KRAS,human,0,43363.0,0.949049,16,1,A431,61.0,2
3,3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,...,KRAS,human,0,23680.0,0.518264,16,0,A431,28.0,3
4,4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,...,KRAS,human,0,41168.0,0.901009,16,0,A431,36.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34760,36885,905505,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,GTTATGAAATTATTGG,...,APOL1,human,0,7827.0,0.541286,16,0,A431,52.0,36876
34761,36886,905634,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,ATTCTATTAGAGGGCT,...,APOL1,human,0,8250.0,0.570539,16,0,A431,29.5,36878
34762,36887,905665,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,GCTTTAAACTCAGGTG,...,APOL1,human,0,8335.0,0.576418,16,0,A431,11.0,36887
34763,36888,972163,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,CGTCAATATATTCTTT,...,APOL1,human,0,13848.0,0.957676,16,0,A431,36.0,36888


In [76]:
cols_to_check = [col for col in df_copy.columns if col not in [INHIBITION, 'index', 'mutual_index', 'corrected_inhibition']]

# Find rows that have duplicates ignoring the 'index' column
dups = df_copy.duplicated(subset=cols_to_check, keep=False)

# Filter those rows, keeping the 'index' column
df_duplicates = df_copy[dups]

In [77]:
df_duplicates_sorted = df_duplicates.sort_values(by=cols_to_check).reset_index(drop=True)

In [83]:
df_duplicates_sorted

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,corrected_inhibition,mutual_index
0,33531,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,2500.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,27.5,33531
1,33579,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,2500.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,27.5,33531
2,34450,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,71.0,34450
3,34470,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,71.0,34450
4,32906,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,20000.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,79.5,32906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,29937,80280157,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,63.5,29776
4925,29777,80280158,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,43.0,29777
4926,29938,80280158,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,43.0,29777
4927,29778,80280159,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,28.5,29778


In [82]:
df_copy.to_csv('data_updated_inhibition.csv', index=False)